In [83]:
import ijson
import json
import numpy as np
import pandas as pd
import os
from pandas.io.json import json_normalize

In [84]:
google_json_path='./inputs/Google/FRIENDS - Hulu (Japan)_celebrity.json'
aws_json_path = './inputs/aws_FRIENDS.json'
azure_json_path = './inputs/azure_FRIENDS.json'

In [85]:
def convert_to_secs(time_str):
    h, m, s = time_str.split(':')
    return int(h)*3600 + int(m)*60 + float(s)

In [86]:
class AzureResponseParser:
    def __init__(self, azure_json_path):
        self.azure_json_path = azure_json_path

    def get_celebrity_info(self):
        with open(self.azure_json_path, 'r') as f:
            celebrities_list = list(ijson.items(f, 'videos.item.insights.faces.item'))
        df = json_normalize(celebrities_list)
        df['source'] = 'microsoft'
        segment_df = df.drop(['description', 'thumbnailId', 'referenceId', 'referenceType', 'title', 'thumbnails'], axis=1)
        segment_df = segment_df.explode('instances').reset_index(drop=True)
        segment_df['segment.start_time'] = segment_df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
        segment_df['segment.end_time'] = segment_df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)
        segment_df = segment_df.rename(columns = {"imageUrl":"url"}).drop('instances', axis=1)
        return segment_df

    def get_logo_info(self):
        pass
    
    def get_label_info(self):
        with open(self.azure_json_path, 'r') as f:
            labels_list = list(ijson.items(f, 'videos.item.insights.labels.item'))
        df = json_normalize(labels_list)
        df['source'] = 'microsoft'
        df = df.explode('instances').reset_index(drop=True)
        df = df.join(json_normalize(df['instances']))
        df['segment.start_time'] = df['start'].apply(lambda x: convert_to_secs(x) if pd.notna(x) else x)
        df['segment.end_time'] = df['end'].apply(lambda x: convert_to_secs(x) if pd.notna(x) else x)
        df.drop(['instances', 'adjustedStart', 'adjustedEnd', 'start', 'end'], axis=1, inplace=True)
        return df
        
    def __add_bounding_box(self, row):
        temp_dict = { 'left': row['left'], \
                      'right': row['left'] + row['width'], \
                      'top' : row['top'], \
                      'bottom' : row['top'] + row['height'] }
        return temp_dict
    
    def get_text_info(self):
        with open(self.azure_json_path, 'r') as f:
            ocr_list = list(ijson.items(f, 'videos.item.insights.ocr.item'))
        df = json_normalize(ocr_list)
        df['source'] = 'microsoft'
        df = df.explode('instances').reset_index(drop=True)
        df['segment.start_time'] = df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
        df['segment.end_time'] = df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)
        df['bounding_box'] = df.apply(self.__add_bounding_box, axis=1)
        df.drop(['instances', 'left', 'top', 'width', 'height'], axis=1, inplace=True)
        return df

    def get_speech_info(self):
        with open(self.azure_json_path, 'r') as f:
            transcript_list = list(ijson.items(f, 'videos.item.insights.transcript.item'))
        df = json_normalize(transcript_list)
        df['source'] = 'microsoft'
        df = df.explode('instances').reset_index(drop=True)
        df['segment.start_time'] = df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
        df['segment.end_time'] = df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)
        df.drop(['instances', 'speakerId'], axis=1, inplace=True)
        return df
        
    def __handle_keyframes(self, input_list):
        out_list = []
        for keyframe in input_list:
            instances = keyframe.pop('instances', None)
            if instances:
                keyframe['start_time'] = convert_to_secs(instances[0]['start'])
                keyframe['end_time'] = convert_to_secs(instances[0]['end'])
        return input_list
    
    def get_shots_info(self):
        with open(self.azure_json_path, 'r') as f:
            shot_list = list(ijson.items(f, 'videos.item.insights.shots.item'))
        df = json_normalize(shot_list)
        df['source'] = 'microsoft'
        df = df.explode('instances').reset_index(drop=True)
        df['segment.start_time'] = df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
        df['segment.end_time'] = df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)
        df['keyframes'] = df['keyFrames'].apply(self.__handle_keyframes)
        df.drop(['instances', 'keyFrames'], axis=1, inplace=True)
        return df
    
    def get_content_moderation_info(self):
        with open(self.azure_json_path, 'r') as f:
            moderation_list = list(ijson.items(f, 'videos.item.insights.visualContentModeration.item'))
        df = json_normalize(moderation_list)
        df['source'] = 'microsoft'
        df = df.explode('instances').reset_index(drop=True)
        df['segment.start_time'] = df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
        df['segment.end_time'] = df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)
        df.drop('instances', axis=1, inplace=True)
        return df


In [87]:
azureParser = AzureResponseParser(azure_json_path)
azure_celeb_info = azureParser.get_celebrity_info()
azure_celeb_info

,id,name,confidence,url,source,segment.start_time,segment.end_time
0,1279,David Schwimmer,1,https://www.bing.com/th?id=AMMS_2400447960bb27...,microsoft,9.259,10.177
1,1279,David Schwimmer,1,https://www.bing.com/th?id=AMMS_2400447960bb27...,microsoft,18.018,20.896
2,1279,David Schwimmer,1,https://www.bing.com/th?id=AMMS_2400447960bb27...,microsoft,23.023,23.983
3,1007,Courteney Cox,0.9986,https://www.bing.com/th?id=AMMS_bc02ff04fe1dee...,microsoft,0.000,1.252
4,1007,Courteney Cox,0.9986,https://www.bing.com/th?id=AMMS_bc02ff04fe1dee...,microsoft,3.337,3.671
5,1007,Courteney Cox,0.9986,https://www.bing.com/th?id=AMMS_bc02ff04fe1dee...,microsoft,18.018,20.896
6,1225,Matt LeBlanc,1,https://www.bing.com/th?id=AMMS_b4adeefb0ac306...,microsoft,0.000,1.252
7,1225,Matt LeBlanc,1,https://www.bing.com/th?id=AMMS_b4adeefb0ac306...,microsoft,11.595,12.471
8,1225,Matt LeBlanc,1,https://www.bing.com/th?id=AMMS_b4adeefb0ac306...,microsoft,13.764,14.932
9,1225,Matt LeBlanc,1,https://www.bing.com/th?id=AMMS_b4adeefb0ac306...,microsoft,26.777,27.528


In [88]:
azure_label_info = azureParser.get_label_info()
azure_label_info

,id,name,referenceId,language,source,confidence,segment.start_time,segment.end_time
0,1,People,person,en-US,microsoft,0.9965,1.335,13.347
1,1,People,person,en-US,microsoft,0.998,14.681,28.028
2,2,Indoor,NaN,en-US,microsoft,0.9422,1.335,6.674
3,2,Indoor,NaN,en-US,microsoft,0.9323,8.008,9.343
4,2,Indoor,NaN,en-US,microsoft,0.963,10.677,28.028
5,3,Outdoor,NaN,en-US,microsoft,0.9243,6.673,8.008
6,3,Outdoor,NaN,en-US,microsoft,0.9881,9.343,10.678
7,4,Wall,structure/wall,en-US,microsoft,0.99,10.677,12.012
8,4,Wall,structure/wall,en-US,microsoft,0.9566,24.024,25.359
9,5,Standing,NaN,en-US,microsoft,0.89,10.677,13.347


In [89]:
azure_text_info = azureParser.get_text_info()
azure_text_info

,id,text,confidence,language,source,segment.start_time,segment.end_time,bounding_box
0,1,"""Friends""",0.9174,en-US,microsoft,11.929,16.058,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
1,1,"""Friends""",0.9174,en-US,microsoft,16.517,16.976,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
2,1,"""Friends""",0.9174,en-US,microsoft,17.434,17.893,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
3,1,"""Friends""",0.9174,en-US,microsoft,18.352,19.269,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
4,1,"""Friends""",0.9174,en-US,microsoft,21.104,22.940,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
5,2,Now being distributed on Hu,0.9324,en-US,microsoft,11.929,16.058,"{'left': 51, 'right': 170, 'top': 50, 'bottom'..."
6,2,Now being distributed on Hu,0.9324,en-US,microsoft,18.352,19.269,"{'left': 51, 'right': 170, 'top': 50, 'bottom'..."
7,2,Now being distributed on Hu,0.9324,en-US,microsoft,21.104,22.940,"{'left': 51, 'right': 170, 'top': 50, 'bottom'..."
8,3,"& CWYo"" ""Day"" 0 Ente ""n nc-",0.5238,en-US,microsoft,13.764,14.223,"{'left': 62, 'right': 256, 'top': 74, 'bottom'..."
9,4,"& eWyo ""Sun ro Ente Le i I i nc-",0.4822,en-US,microsoft,14.223,15.140,"{'left': 62, 'right': 256, 'top': 74, 'bottom'..."


In [90]:
azure_speech_info = azureParser.get_speech_info()
azure_speech_info

,id,text,confidence,language,source,segment.start_time,segment.end_time
0,1,Right.,1,en-US,microsoft,0.57,4.629
1,2,SNS This is Tsukuba Iceland.,1,en-US,microsoft,5.38,30.030


In [91]:
azure_shots_info = azureParser.get_shots_info()
azure_shots_info

,id,tags,source,segment.start_time,segment.end_time,keyframes
0,1,"[Wide, Indoor]",microsoft,0.000,2.753,"[{'id': 1, 'start_time': 0.25, 'end_time': 0.2..."
1,2,"[Wide, CenterFace, Indoor, Medium]",microsoft,2.753,5.714,"[{'id': 3, 'start_time': 2.753, 'end_time': 2...."
2,3,"[Indoor, Medium]",microsoft,5.714,6.840,"[{'id': 5, 'start_time': 5.714, 'end_time': 5...."
3,4,"[Medium, RightFace, Outdoor]",microsoft,6.840,8.926,"[{'id': 7, 'start_time': 6.84, 'end_time': 6.8..."
4,5,[Indoor],microsoft,8.926,10.219,"[{'id': 9, 'start_time': 8.926, 'end_time': 8...."
5,6,"[Medium, CenterFace, Outdoor]",microsoft,10.219,11.887,"[{'id': 10, 'start_time': 10.219, 'end_time': ..."
6,7,"[Wide, Indoor]",microsoft,11.887,13.639,"[{'id': 11, 'start_time': 11.887, 'end_time': ..."
7,8,"[Indoor, Medium, CenterFace]",microsoft,13.639,14.932,"[{'id': 13, 'start_time': 13.639, 'end_time': ..."
8,9,"[Medium, CenterFace, Indoor, Wide]",microsoft,14.932,18.060,"[{'id': 15, 'start_time': 14.932, 'end_time': ..."
9,10,"[Medium, Indoor]",microsoft,18.060,20.938,"[{'id': 17, 'start_time': 18.06, 'end_time': 1..."


In [92]:
azure_moderation_info = azureParser.get_content_moderation_info()
azure_moderation_info

,id,adultScore,racyScore,source,segment.start_time,segment.end_time
0,1,0.0024,0.9296,microsoft,13.931,13.973


In [93]:
restructured_json = {'celebrities': json.loads(azure_celeb_info.to_json(orient='records')),
                     'labels': json.loads(azure_label_info.to_json(orient='records')),
                     'texts': json.loads(azure_text_info.to_json(orient='records')),
                     'speech': json.loads(azure_speech_info.to_json(orient='records')),
                     'shots': json.loads(azure_shots_info.to_json(orient='records')),
                     'content_moderation': json.loads(azure_moderation_info.to_json(orient='records'))
                    }
with open('restructured_azure.json', "w") as write_file:
    json.dump(restructured_json, write_file, indent=4)